In [7]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from basic_SVD import find_SVD
from basic_SVD import recombine_SVD


In [ ]:
img_path = "fish.jpeg"
image_array = np.array(Image.open(img_path))
image_R = image_array[:, :, 0]  # Red channel
image_G = image_array[:, :, 1]  # Green channel
image_B = image_array[:, :, 2]  # Blue channel
image_grayscale = np.dot(image_array[..., :3], [0.2989, 0.5870, 0.1140])
# plt.imshow(image_grayscale, cmap='gray')
plt.imshow(image_array)

In [9]:
U, S, V = find_SVD(image_grayscale)

U_r, S_r, V_r = find_SVD(image_R)
U_g, S_g, V_g = find_SVD(image_G)
U_b, S_b, V_b = find_SVD(image_B)

m = image_array.shape[0]
n = image_array.shape[1]
# reduced_image = recombine_SVD(U, S, V, 50)
# plt.imshow(reduced_image, cmap="gray")

In [ ]:


# Create a slider
slider = widgets.IntSlider(
    value=50,          # Initial value
    min=1,             # Minimum value
    max=min(m,n),           # Maximum value
    step=1,            # Step size
    description='Value:',  # Label for the slider
    continuous_update=True, # Update the value continuously as the slider moves
    layout=widgets.Layout(width='60%')
)

checkbox = widgets.Checkbox(
    value=False,  # Default value (unchecked)
    description='Colored',  # Label for the checkbox
    disabled=False  # If True, the checkbox will be grayed out
)

def show_reduced_gray_image(k):
    image = recombine_SVD(U, S, V, k)
    plt.imshow(image, cmap='gray')
    plt.axis('off')  # Turn off the axes
    plt.show()

def show_reduced_color_image(k):
    image_r = recombine_SVD(U_r, S_r, V_r, k).astype(np.uint8)
    image_g = recombine_SVD(U_g, S_g, V_g, k).astype(np.uint8)
    image_b = recombine_SVD(U_b, S_b, V_b, k).astype(np.uint8)
    image_rgb = np.stack((image_r, image_g, image_b), axis=-1)
    plt.imshow(image_rgb, cmap='gray')
    plt.axis('off')  # Turn off the axes
    plt.show()

def show_reduced_image(k, isColor):
    if isColor:
        show_reduced_color_image(k)
    else:
        show_reduced_gray_image(k)

# Display the slider
interactive_plot = widgets.interactive(show_reduced_image, k=slider, isColor=checkbox)
display(interactive_plot)